## Петренко Дарья, группа 317
## сравнение реализаций функции calc_expectations 

In [1]:
import numpy as np
from scipy.signal import convolve2d
from random import random, randint


row_num = 50
col_num = 100

1) полностью невекторизованная реализация

In [2]:
def calc_expectations_non_vect(h, w, X, Q):
    prob_matr = list()
    for i in range(len(X)):
        curr_list = list()
        for j in range(len(X[0])):
            sum = 0
            for m in range(max(i - h + 1, 0), i + 1):
                for n in range(max(j - w + 1, 0), j + 1):
                    sum += Q[m][n]
            curr_list.append(sum)
        prob_matr.append(curr_list)
    for i in range(len(X)):
        for j in range(len(X[0])):
            prob_matr[i][j] *= X[i][j]
    return prob_matr

2) частично векторизованная реализация

In [3]:
def calc_expectations_half_vect(h, w, X, Q):
    row_num = np.size(X, 0)  
    col_num = np.size(X, 1)
    prob_matr = np.empty((row_num, col_num))
    for i in range(row_num):
        for j in range(col_num):
            prob_matr[i, j] = np.sum(Q[max(i - h + 1, 0):i + 1, max(j - w + 1, 0):j + 1])
    return X * prob_matr

3) полностью векторизованная реализация

In [4]:
def calc_expectations_vect(h, w, X, Q):
    conv_matr = np.ones((h, w))
    Q = convolve2d(Q, conv_matr, mode='full')
    Q = Q[:np.size(X, 0), :np.size(X, 1)]
    return X * Q

In [5]:
X = []
Q = []

for i in range(row_num):
    curr_list_X = []
    curr_list_Q = []
    for j in range(col_num):
        curr_list_X.append(randint(0, 100))
        curr_list_Q.append(random())
    X.append(curr_list_X)
    Q.append(curr_list_Q)
    
X_np = np.array(X)
Q_np = np.array(Q)

In [6]:
%timeit calc_expectations_non_vect(2, 2, X, Q)

9.09 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit calc_expectations_half_vect(2, 2, X_np, Q_np)

26.6 ms ± 2.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit calc_expectations_vect(2, 2, X_np, Q_np)

137 µs ± 600 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Выводы: Как и ожидалось, полностью векторизованная функция работает намного быстрее остальных. Однако частично векторизованная функция в несколько раз медленнее невекторизованной, причем тенденция сохраняется для матриц различных размеров, от 2 до нескольких сотен по каждому измерению. У меня есть предположение, что структура данных numpy array неэффективно работает с некоторыми базовыми функциями языка Питон. Ниже показано, что простое итерирование по всем элементам массива типа numpy array с суммированием всех элементов в несколько раз медленнее тех же действий, произведенных с двумерным списком.

In [9]:
def my_loop_np():
    sum = 0
    for i in range(row_num):
        for j in range(col_num):
            sum += Q_np[i][j]

In [10]:
def my_loop():
    sum = 0
    for i in range(row_num):
        for j in range(col_num):
            sum += Q[i][j]

In [11]:
%timeit my_loop_np()

1.44 ms ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit my_loop()

366 µs ± 4.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
